In [157]:
import gzip
import numpy
from collections import defaultdict

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

In [158]:
Data = []
for l in readGz("assignment1/train.json.gz"):
    Data.append(l)

In [159]:
data_train = Data[:100000]
data_valid = Data[100000:]

In [160]:
# task 1
alpha = 0
count = 0
for d in data_train:
    if d['helpful']['outOf'] != 0:
        alpha += d['helpful']['nHelpful'] * 1.0 / d['helpful']['outOf']
        count += 1

alpha /= count
print alpha

0.783737364232


In [161]:
# task 2
MAE = 0
for d in data_valid:
    predict = alpha * d['helpful']['outOf']
    MAE += abs(predict - d['helpful']['nHelpful'])

MAE /= len(data_valid)
print MAE

0.258702157422


In [162]:
# task 3
def filter(data):
    feature= []
    label = []
    for d in data:
        if d['helpful']['outOf'] != 0:
            # feature
            feat = [1]
            feat.append(len(d['reviewText'].strip().split()))
            feat.append(d['rating'])
            feature.append(feat)
            # label
            l = d['helpful']['nHelpful'] * 1.0 / d['helpful']['outOf']
            label.append(l)
    return feature, label

In [163]:
feature_train, label_train = filter(data_train)
theta,residuals,rank,s = numpy.linalg.lstsq(feature_train, label_train)

In [164]:
print theta

[  5.62218966e-01   2.11835412e-04   5.07029148e-02]


In [165]:
# task 3 MAE
def inner(x, y): 
    result = 0
    for a, b in zip(x, y): 
        result += a * b
    return result

feature_valid, label_valid = filter(data_valid)
helpful_valid = []
for d in data_valid:
    if d['helpful']['outOf'] != 0:
        helpful_valid.append(d['helpful'])
        
MAE = 0
for i in range(len(helpful_valid)):
    predict = inner(theta, feature_valid[i]) * helpful_valid[i]['outOf']
    MAE += abs(helpful_valid[i]['nHelpful'] - predict) 
MAE /= len(data_valid)
print MAE

0.240245808704


In [166]:
# task 4
reviews = {}
for l in readGz("assignment1/test_Helpful.json.gz"):
    user,item = l['reviewerID'], l['itemID']
    key = user + ' ' + item
    reviews[key] = {}
    reviews[key]["rating"] = l["rating"]
    reviews[key]["outOf"] = l["helpful"]["outOf"]
    reviews[key]["length"] = len(l['reviewText'].strip().split())

In [167]:
predictions = open("assignment1/predictions_Helpful.txt", 'w')

for l in open("assignment1/pairs_Helpful.txt"): 
    if l.startswith("userID"):
        # first line
        predictions.write(l)
        continue
    
    u, i, outOf = l.strip().split('-')
    outOf = int(outOf)
    key = u + ' ' + i
    feature = [1, reviews[key]["length"], reviews[key]["rating"]]
    predictions.write(u + '-' + i + '-' + str(outOf) + ',' \
                      + str(outOf*(inner(theta , feature)))+ '\n')

predictions.close ()